In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import json
import random
import re

model = '355M'

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
!nvidia-smi

Mon Dec 30 07:33:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    39W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
gpt2.download_gpt2(model_name=model)

Fetching checkpoint: 1.05Mit [00:00, 425Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 122Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 745Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:09, 156Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 460Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 135Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 195Mit/s]                                                       


In [0]:
import json

with open('/content/drive/My Drive/Colab Notebooks/data/trumpprestweets.json', encoding='utf-8') as f:
    data = json.load(f)

In [0]:
s = []
random.shuffle(data)
for i in data:
    if 'is_retweet' not in i.keys() or not i['is_retweet']:
        text = str(i['text'])
        text = re.sub(r"http\S+", "", text)
        text = text.replace('\n','')
        text = text.strip()
        if text[:2] == '"@' or text[:2] == '..' or text[-2:] == '..':
            text = ""
        if not text == '':
            s.append(text)

In [0]:
import pandas as pd

p = pd.DataFrame()

p[1] = s

p.to_csv("trumptweets.csv", index = False, header = False)

In [0]:
p

,1
0,President Obama said that he thinks he would h...
1,FBI &amp; DOJ to review the outrageous Jussie ...
2,.@RichLowry is truly one of the dumbest of the...
3,Why would anyone in Florida vote for lightweig...
4,"Happy Birthday Regis, a truly special man!"
...,...
13366,Thank you to all of our law enforcement office...
13367,"Moderator: Hillary paid $225,000 by a Brazilia..."
13368,After @TrumpTurnberry I will be visiting Aberd...
13369,Last night in Minnesota was incredible. What a...


In [0]:
!du -h trumptweets.csv

1.8M	trumptweets.csv


In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset='trumptweets.csv',
              model_name=model,
              steps=2000,
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=100,
              save_every=1000
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 19.08it/s]

Loading dataset...


dataset has 616801 tokens
Training...
[10 | 16.82] loss=2.28 avg=2.28
[20 | 25.68] loss=2.37 avg=2.33
[30 | 34.53] loss=2.53 avg=2.40
[40 | 43.37] loss=2.04 avg=2.31
[50 | 52.21] loss=1.81 avg=2.21
[60 | 61.05] loss=2.37 avg=2.23
[70 | 69.89] loss=1.89 avg=2.18
[80 | 78.73] loss=2.08 avg=2.17
[90 | 87.60] loss=1.95 avg=2.14
[100 | 96.45] loss=2.05 avg=2.13
======== SAMPLE 1 ========
IN<|endoftext|>
<|startoftext|>My job is a great one – get paid fairly and fairly. You are very lucky!<|endoftext|>
<|startoftext|>“DNC” is in the hole.   The #DrainTheSwamp will grow fast, and that will make this great! The Democrats are in a total con! They are in total trouble, and to hell with them. If they will do something, the nation will reward them!<|endoftext|>
<|startoftext|>If you think President Trump would be the best or person to lead, or even run, for President of the United States, think again. The people would let loose. MAKE AMERICA GREAT AGAIN!<|endoftext|>
<|startoftext|>Dump the Crooke

In [0]:
text = gpt2.generate(sess, nsamples = 10, batch_size = 10, prefix='<|startoftext|>', include_prefix=False, truncate='<|endoftext|>', return_as_list=True)

NameError: ignored

In [0]:
for i in text:
  print(i)

NameError: ignored

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')